 [코알라] 주식 종료 가격 예측 경진대회 Stage 5 : 모델 소개 및 학습

안녕하세요. 데이크루 4기 코알라🐨 팀입니다. 

저희는 📈주식 종료 가격 예측 경진대회를 주제로 PBL를 수행합니다.

이번 활동을 통해 논리적인 접근 방식으로 모든 문제를 풀어갈 수 있는 데이커가 되는 것을 최종 목표로 하고 있습니다. 

Stage 5은 PBL을 구성하는 6가지 단계 중 5번째 단계로서 모델 소개 및 학습 내용을 담고있습니다.

다음의 포스팅은 데이크루 4기 활동으로 인하여 작성되었음을 알려드립니다.


# Stage 5.
# 1. Review

Stage 5의 코드를 정상적으로 실행하기 위해서는, 앞서 Stage 4에서 진행했던 코드를 전부 실행한 상태여야 합니다.

뒤에서 사용할 기본 설정을 위해 저희가 추가적인 코드와 함께 한번에 준비했으니, 이 코드창을 실행시키고 들어가주세요!


In [ ]:
import micropip
await micropip.install('requests')

In [3]:
import pandas as pd
import numpy as np
import datetime
import requests as req
import lightgbm as lgb
from sklearn import linear_model
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from sklearn import metrics
import os
import sys
import random
import warnings

In [4]:
warnings.filterwarnings(action='ignore')
os.chdir('./practice_data')

In [5]:
## 과거 상장폐지종목 추출 
df_out_tt1 = pd.read_csv('코스피상폐현황.csv', encoding='utf-8')
df_out_tt2 = pd.read_csv('코스닥상폐현황.csv', encoding='utf-8')

df_out_tt1['Market'] = 'KOSPI'
df_out_tt2['Market'] = 'KOSDAQ'
df_out_tt = pd.concat([df_out_tt1, df_out_tt2])
df_out_tt.rename(columns = {"종목코드":"code", "회사명":"code_nm", "폐지일자":"out_dt", "폐지사유":"out_desc", "Market":'market'}, inplace = True)
df_out_tt['code'] = df_out_tt['code'].astype(str) 
df_out_tt['code'] = df_out_tt['code'].str.zfill(6) 
df_out_tt['out_dt'] = pd.to_datetime(df_out_tt['out_dt'])
df_out_tt = df_out_tt[["code", "code_nm", "out_dt", "out_desc", "market"]]

## 현재 상장종목 전체 추출 : FinanceDataReader
df_in_tt = pd.read_csv('종목리스트.csv', encoding='utf-8-sig')
df_in_tt.columns = ['code', 'isu_cd', 'code_nm', 'market', 'dept', 'close', 'changecode', 'changes', 'chagesratio', 
                    'open', 'high', 'low', 'volume', 'amount', 'marcap', 'stocks', 'marketId']
df_in_tt = df_in_tt[(df_in_tt['market'] !='KONEX') &                
                    (df_in_tt.code.str.startswith('5') != True) &   
                    (df_in_tt.code.str.startswith('6') != True) &   
                    (df_in_tt.code.str.startswith('7') != True) &   
                    (df_in_tt.code.str.len() == 6)]
df_in_tt['in_yn'] = 1  
df_in_tt['code'] = df_in_tt['code'].astype(str) 
df_in_tt['code'] = df_in_tt['code'].str.zfill(6) 
df_in_tt['close'] = df_in_tt['close'].astype(str)
df_in_tt['changecode'] = df_in_tt['close'].astype(str)

## 과거 상장폐지종목 + 현재 상장종목 결합
end_dt = datetime.datetime.now().strftime('%Y%m%d')   
df_stock_code = pd.merge(df_out_tt, df_in_tt, how='outer', on='code', suffixes=('_old', '_new'))
df_stock_code['in_yn'] = np.where(df_stock_code['in_yn'].notnull(), df_stock_code['in_yn'], 0)
df_stock_code['market'] = np.where(df_stock_code['market_new'].notnull(), df_stock_code['market_new'], df_stock_code['market_old'])
df_stock_code['code_nm'] = np.where(df_stock_code['code_nm_new'].notnull(), df_stock_code['code_nm_new'], df_stock_code['code_nm_old'])
df_stock_code['base_dt'] = pd.to_datetime(end_dt)
df_stock_code.rename(columns = {"ListingDate":"in_dt"}, inplace = True) 
df_stock_code = df_stock_code.sort_values(by=['code', 'base_dt'], axis=0, ascending=[True, False])  
df_stock_code = df_stock_code.drop_duplicates(['code'], keep='first')
df_stock_code = df_stock_code[['code','code_nm','in_yn','market','out_dt','out_desc','base_dt']]
df_stock_code = df_stock_code[df_stock_code['code']=='005930']

df_price = pd.read_csv('주가파일.csv', encoding='utf-8')
df_price['dt'] = pd.to_datetime(df_price['dt'])
df_price['dt_base'] = pd.to_datetime(df_price['dt_base'])
df_price['code'] = df_price['code'].astype(str)
df_price['code'] = df_price['code'].str.zfill(6)

df_usdkrw = pd.read_csv('환율파일.csv', encoding='utf-8')    
df_index = pd.read_csv('주가지수.csv', encoding='utf-8')
df_index['dt'] = pd.to_datetime(df_index['dt'])
df_usdkrw['dt'] = pd.to_datetime(df_usdkrw['dt'])

## 학습 대상 종목 선정 
stock_list = pd.read_csv("./stock_list.csv")
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list.rename(columns = {"종목코드":"code"}, inplace = True) 

df_master = df_price[(df_price['open'] > 0) & (df_price['end'] > 0) & (df_price['cnt'] > 0)]
df_master['amt']  = df_master['end'] * df_master['cnt']   
df_amt = df_master.groupby(['code'], as_index=False)[['end', 'cnt', 'amt']].mean()
df_amt = df_amt[(df_amt['end'] >= 3000) & (df_amt['amt'] >= 2000000000)]     
dl_amt = df_amt['code'].values.tolist() 
dl_200 = stock_list['code'].values.tolist() 
dl_amt = set(dl_amt + dl_200)
df_master = df_master[df_master['code'].isin(dl_amt)]


In [6]:
## Target 생성
df_master = df_master.sort_values(by=['code', 'dt'], axis=0, ascending=[True, False])  
df_master['num'] = df_master.groupby('code')['dt'].rank(ascending=True).astype(int)
df_master['y_01d_yn']  = np.where(df_master['code'] == df_master['code'].shift(1), 1, 0)  
df_master['y_02d_yn']  = np.where(df_master['code'] == df_master['code'].shift(2), 1, 0)  
df_master['y_03d_yn']  = np.where(df_master['code'] == df_master['code'].shift(3), 1, 0)  
df_master['y_04d_yn']  = np.where(df_master['code'] == df_master['code'].shift(4), 1, 0)  
df_master['y_05d_yn']  = np.where(df_master['code'] == df_master['code'].shift(5), 1, 0)  
df_master['y_rt_01d_eeup']  = df_master['end'].shift(1) / df_master['end']
df_master['y_rt_02d_eeup']  = df_master['end'].shift(2) / df_master['end']
df_master['y_rt_03d_eeup']  = df_master['end'].shift(3) / df_master['end']
df_master['y_rt_04d_eeup']  = df_master['end'].shift(4) / df_master['end']
df_master['y_rt_05d_eeup']  = df_master['end'].shift(5) / df_master['end']

df_master = df_master.sort_values(by=['code', 'dt'], axis=0) 
df_master = pd.merge(df_master, df_index[['dt', 'end_kpi', 'end_ksd']], how='left', on='dt')
df_master = pd.merge(df_master, df_stock_code[['code', 'market']], how='left', on='code')
df_master['x_rt_end_index']  = np.where(df_master['market'] == 'KOSPI', df_master['end'] / df_master['end_kpi'], 
                                        df_master['end'] / df_master['end_ksd'])
df_master = pd.merge(df_master, df_usdkrw[['dt', 'end_usdkrw']], how='left', on='dt')
df_master['x_rt_end_usdkrw'] = df_master['end'] / df_master['end_usdkrw']

## 시/저/고/저 파생
df_master['avg_amt_20'] = df_master['amt'].rolling(window=20).mean() 
df_master['x_rt_h_l']  = np.where(df_master['low'] == 0,  np.nan, df_master['high'] / df_master['low'])      
df_master['x_rt_e_s']  = np.where(df_master['open'] == 0,  np.nan, df_master['end'] / df_master['open'])   
df_master['x_rt_e_l']  = np.where(df_master['low'] == 0,  np.nan, df_master['end'] / df_master['low']) 
df_master['x_rt_h_s']  = np.where(df_master['open'] == 0,  np.nan, df_master['high'] / df_master['open'])   
df_master['x_rt_s_l']  = np.where(df_master['low'] == 0,  np.nan, df_master['open'] / df_master['low'])  
df_master['x_rt_bf_1']  = np.where(df_master['end'].shift(1) == 0,   np.nan, df_master['end'] / df_master['end'].shift(1))


In [7]:
## 과거 평균 대비 이격도 Feature 
k  = 3; k2 = 10; k3 = 40; k4 = 80

dl_num_cols = ['open', 'high', 'low', 'end', 'amt', 'end_kpi', 'end_ksd', 'x_rt_end_index',
               'x_rt_bf_1', 'x_rt_e_s', 'x_rt_e_l', 'end_usdkrw', 'x_rt_end_usdkrw' ] 

dl_cols =[('x_rt_bf_1_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].shift(1) == 0,  np.nan, df_master[dl_num_cols] / df_master[dl_num_cols].shift(1))

dl_cols =[('x_rt_ma_k_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k).mean() == 0,  np.nan, 
                      (df_master[dl_num_cols]-df_master[dl_num_cols].rolling(window=k).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols] >= df_master[dl_num_cols].rolling(window=k).mean(), 
                      abs(df_master[dl_cols]), -abs(df_master[dl_cols]))                                                
                        
dl_cols =[('x_rt_ma_k2_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k2).mean() == 0,  np.nan, 
                      (df_master[dl_num_cols]-df_master[dl_num_cols].rolling(window=k2).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols] >= df_master[dl_num_cols].rolling(window=k2).mean(), 
                      abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
                        
dl_cols =[('x_rt_ma_k_k2_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k2).mean() == 0,  np.nan, 
                      (df_master[dl_num_cols].rolling(window=k).mean()-df_master[dl_num_cols].rolling(window=k2).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k).mean() >= df_master[dl_num_cols].rolling(window=k2).mean(), 
                      abs(df_master[dl_cols]), -abs(df_master[dl_cols]))

dl_cols =[('x_rt_ma_k3_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k3).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols]-df_master[dl_num_cols].rolling(window=k3).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols] >= df_master[dl_num_cols].rolling(window=k3).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k_k3_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k3).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k).mean()-df_master[dl_num_cols].rolling(window=k3).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k).mean() >= df_master[dl_num_cols].rolling(window=k3).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k2_k3_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k3).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k2).mean()-df_master[dl_num_cols].rolling(window=k3).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k2).mean() >= df_master[dl_num_cols].rolling(window=k3).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))							 
							 
dl_cols =[('x_rt_ma_k4_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k4).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols]-df_master[dl_num_cols].rolling(window=k4).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols] >= df_master[dl_num_cols].rolling(window=k4).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k_k4_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k4).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k).mean()-df_master[dl_num_cols].rolling(window=k4).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k).mean() >= df_master[dl_num_cols].rolling(window=k4).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k2_k4_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k4).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k2).mean()-df_master[dl_num_cols].rolling(window=k4).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k2).mean() >= df_master[dl_num_cols].rolling(window=k4).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))
							 
dl_cols =[('x_rt_ma_k3_k4_' + w) for w in dl_num_cols]
df_master[dl_cols]  = np.where(df_master[dl_num_cols].rolling(window=k4).mean() == 0,  np.nan, 
							 (df_master[dl_num_cols].rolling(window=k3).mean()-df_master[dl_num_cols].rolling(window=k4).mean()) / df_master[dl_num_cols].rolling(window=k2).mean())  
df_master[dl_cols] = np.where(df_master[dl_num_cols].rolling(window=k3).mean() >= df_master[dl_num_cols].rolling(window=k4).mean(), 
							 abs(df_master[dl_cols]), -abs(df_master[dl_cols]))

## 단순이동평균 대비 지수이동평균 대비 이격도
for col in dl_num_cols:
	str_ma05 = 'ma05_' + col
	str_ema05 = 'ema05_' + col
	str_rt_ema05_ma05 = 'x_rt_ema05_ma05_' + col
	df_master[str_ma05] = df_master[col].rolling(window=5).mean() 		
	df_master[str_ema05]  = df_master[col].shift(5) 
	tmp1 = 0.7 	
	for i in range(5):
		df_master[str_ema05]  = (df_master[col].shift(5-1-i) * tmp1) + (df_master[str_ema05] * (1-tmp1))		
	df_master[str_rt_ema05_ma05] = np.where(df_master[str_ma05] == 0,  np.nan, (df_master[str_ema05]-df_master[str_ma05]) / df_master[str_ma05])  
	df_master[str_rt_ema05_ma05] = np.where(df_master[str_ema05] >= df_master[str_ma05], abs(df_master[str_rt_ema05_ma05]), -abs(df_master[str_rt_ema05_ma05]))		   

	str_ma20 = 'ma20_' + col
	str_ema20 = 'ema20_' + col
	str_rt_ema20_ma20 = 'x_rt_ema20_ma20_' + col
	df_master[str_ma20] = df_master[col].rolling(window=20).mean() 		
	df_master[str_ema20]  = df_master[col].shift(20) 
	tmp1 = 0.7 	
	for i in range(20):
		df_master[str_ema20]  = (df_master[col].shift(20-1-i) * tmp1) + (df_master[str_ema20] * (1-tmp1))		
	df_master[str_rt_ema20_ma20] = np.where(df_master[str_ma20] == 0,  np.nan, (df_master[str_ema20]-df_master[str_ma20]) / df_master[str_ma20])  
	df_master[str_rt_ema20_ma20] = np.where(df_master[str_ema20] >= df_master[str_ma20], abs(df_master[str_rt_ema20_ma20]), -abs(df_master[str_rt_ema20_ma20]))		   

	str_ma40 = 'ma40_' + col
	str_ema40 = 'ema40_' + col
	str_rt_ema40_ma40 = 'x_rt_ema40_ma40_' + col
	df_master[str_ma40] = df_master[col].rolling(window=40).mean() 		
	df_master[str_ema40]  = df_master[col].shift(40) 
	tmp1 = 0.7 	
	for i in range(40):
		df_master[str_ema40]  = (df_master[col].shift(40-1-i) * tmp1) + (df_master[str_ema40] * (1-tmp1))		
	df_master[str_rt_ema40_ma40] = np.where(df_master[str_ma40] == 0,  np.nan, (df_master[str_ema40]-df_master[str_ma40]) / df_master[str_ma40])  
	df_master[str_rt_ema40_ma40] = np.where(df_master[str_ema40] >= df_master[str_ma40], abs(df_master[str_rt_ema40_ma40]), -abs(df_master[str_rt_ema40_ma40]))		   

	str_ma80 = 'ma80_' + col
	str_ema80 = 'ema80_' + col
	str_rt_ema80_ma80 = 'x_rt_ema80_ma80_' + col
	df_master[str_ma80] = df_master[col].rolling(window=80).mean() 		
	df_master[str_ema80]  = df_master[col].shift(80) 
	tmp1 = 0.7 	
	for i in range(80):
		df_master[str_ema80]  = (df_master[col].shift(80-1-i) * tmp1) + (df_master[str_ema80] * (1-tmp1))		
	df_master[str_rt_ema80_ma80] = np.where(df_master[str_ma80] == 0,  np.nan, (df_master[str_ema80]-df_master[str_ma80]) / df_master[str_ma80])  
	df_master[str_rt_ema80_ma80] = np.where(df_master[str_ema80] >= df_master[str_ma80], abs(df_master[str_rt_ema80_ma80]), -abs(df_master[str_rt_ema80_ma80]))


In [8]:
## Lag 변수		
dl_code = ['x_rt_bf_1', 'x_rt_h_l', 'x_rt_e_s', 'x_rt_e_l']
for m in range(10):   
	for code in dl_code:
		str_x = 'x_lag_' + code + str(m+1) 	
		df_master[str_x] = df_master[code].shift(m+1)
	

## NVI(Negative Volume Index)

# end + cnt	
df_master['x_ind_nvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10)
df_master['end_tmp'] = df_master['end'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['end_tmp'] == 0, np.nan, 
								   ((df_master['end'].shift(i) - df_master['end_tmp']) / df_master['end_tmp']) * 100)
	df_master['x_ind_nvi_10'] = np.where(df_master['cnt'].shift(i) < df_master['cnt_tmp'], df_master['x_ind_nvi_10'] + df_master['tmp1'], 
										df_master['x_ind_nvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['end_tmp'] = df_master['end'].shift(i)
	
# high + cnt	
df_master['x_ind_h_nvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10)
df_master['high_tmp'] = df_master['high'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['high_tmp'] == 0, np.nan, 
								   ((df_master['high'].shift(i) - df_master['high_tmp']) / df_master['high_tmp']) * 100)
	df_master['x_ind_h_nvi_10'] = np.where(df_master['cnt'].shift(i) < df_master['cnt_tmp'], df_master['x_ind_h_nvi_10'] + df_master['tmp1'], 
										df_master['x_ind_h_nvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['high_tmp'] = df_master['high'].shift(i)
	

# low + cnt	
df_master['x_ind_l_nvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10)
df_master['low_tmp'] = df_master['low'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['low_tmp'] == 0, np.nan, 
								   ((df_master['low'].shift(i) - df_master['low_tmp']) / df_master['low_tmp']) * 100)
	df_master['x_ind_l_nvi_10'] = np.where(df_master['cnt'].shift(i) < df_master['cnt_tmp'], df_master['x_ind_l_nvi_10'] + df_master['tmp1'], 
										df_master['x_ind_l_nvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['low_tmp'] = df_master['low'].shift(i)
		
	
## PVI(Positive Volume Index)

# end + cnt										
df_master['x_ind_pvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10) 
df_master['end_tmp'] = df_master['end'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['end_tmp'] == 0, np.nan, 
								   ((df_master['end'].shift(i) - df_master['end_tmp']) / df_master['end_tmp']) * 100)
	df_master['x_ind_pvi_10'] = np.where(df_master['cnt'].shift(i) > df_master['cnt_tmp'], df_master['x_ind_pvi_10'] + df_master['tmp1'], 
										df_master['x_ind_pvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['end_tmp'] = df_master['end'].shift(i)
	
# high + cnt										
df_master['x_ind_h_pvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10) 
df_master['high_tmp'] = df_master['high'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['high_tmp'] == 0, np.nan, 
								   ((df_master['high'].shift(i) - df_master['high_tmp']) / df_master['high_tmp']) * 100)
	df_master['x_ind_h_pvi_10'] = np.where(df_master['cnt'].shift(i) > df_master['cnt_tmp'], df_master['x_ind_h_pvi_10'] + df_master['tmp1'], 
										df_master['x_ind_h_pvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['high_tmp'] = df_master['high'].shift(i)
	
# low + cnt										
df_master['x_ind_l_pvi_10'] = 100
df_master['cnt_tmp'] = df_master['cnt'].shift(10) 
df_master['low_tmp'] = df_master['low'].shift(10) 
for i in range(9, -1, -1):  
	df_master['tmp1'] = np.where(df_master['low_tmp'] == 0, np.nan, 
								   ((df_master['low'].shift(i) - df_master['low_tmp']) / df_master['low_tmp']) * 100)
	df_master['x_ind_l_pvi_10'] = np.where(df_master['cnt'].shift(i) > df_master['cnt_tmp'], df_master['x_ind_l_pvi_10'] + df_master['tmp1'], 
										df_master['x_ind_l_pvi_10'])
	df_master['cnt_tmp'] = df_master['cnt'].shift(i)
	df_master['low_tmp'] = df_master['low'].shift(i)

## Rule Feature
df_master['x_cls_01'] = np.where((df_master['open'] < df_master['end']) & (df_master['high'] > df_master['end']) & (df_master['high']/df_master['end'].shift(1) <=1.01), 1,
					    np.where((df_master['open'] < df_master['end']) & (df_master['high'] > df_master['end']), 2, 3))
						
dl_list = df_master.columns[pd.Series(df_master.columns).str.startswith('x_cls_')]   
df_master[dl_list] = df_master[dl_list].astype(str) 


# 2. 모델 소개

## LightGBM
LightGBM(이하 LGBM)은 트리 기반의 학습 알고리즘인 gradient boosting 방식의 프레임 워크입니다.

LGBM은 유사한 알고리즘인 XGBoost에 비해 훈련 시간이 짧고 성능도 좋아 부스팅 알고리즘에서 가장 많은 주목을 받고 있습니다. 

LGBM의 경우 복잡한 것은 파라미터 튜닝입니다. 

LGBM은 100개 이상의 파라미터를 가지고 있습니다. 그렇기 때문에 LGBM의 가장 베이직한 파라미터를 아는 것이 구현 시 매우 중요합니다. 


## LGBM의 대표적인 파라미터

- **n_leaves** : 전체 Tree의 leave 수 이고, 디폴트값은 31입니다.

- **n_estimators** : 생성할 weak learner의 수이고, 너무 크면 과대적합이 발생합니다.

- **max_depth** :  Tree의 최대 깊이를 말합니다. 이 파라미터는 모델 과적합을 다룰 때 사용됩니다. 만약 모델이 과적합된 것 같다고 느끼신다면 max_depth 값을 줄이는 것을 추천드립니다.

- **learning_rate** : 최종 결과에 대한 각각의 Tree에 영향을 미치는 변수입니다. GBM은 초기의 추정값에서 시작하여 각각의Tree 결과를 사용하여 추정값을 업데이트 합니다. 학습 파라미터는 이러한 추정에서 발생하는 변화의 크기를 컨트롤합니다. 일반적인 값은 0.1, 0.001, 0.003 등등이 있습니다.

- **subsample** : weak learner가 학습에 사용하는 데이터 샘플링 비율입니다. 보통 0.5~ 1이 사용되며 값이 낮을수록 과적합이 방지됩니다.

- **colsample_bytree** : 컬럼을 랜덤으로 추출하여 사용해 각각의 다양성을 높입니다. 랜덤포레스트에 있는 기능이며, 보통 정확도를 높이는 효과가 있습니다.

- **reg_alpha** : 가중치에 대한 L1 Regularization 적용 값입니다. 피처 개수가 많을 때 적용을 검토하며 이 값이 클 수록 과적합 감소 효과가 있습니다. 기본값은 0입니다.

- **reg_lambda** : 가중치에 대한 L2 Regularization 적용 값입니다. 피처 개수가 많을 때 적용을 검토하며 이 값이 클 수록 과적합 감소 효과가 있습니다. 기본값은 0입니다.

- **objective** : 목적(손실)함수로 이 함수의 결과값이 최소화되는 방향으로 학습한다. 회귀인지, 분류인지 등에 따라 옵션을 다르게 선택해야 한다. regression, binary, multicalss 모두 가능합니다.

- **min_data_in_leaf** : Leaf가 가지고 있는 최소한의 레코드 수입니다. 디폴트값은 20으로 최적 값입니다. 과적합을 해결할 때 사용되는 파라미터입니다.

- **random_state** :  랜덤 시드 고정 값으로서, 반드시 고정해두고 튜닝해야 합니다.

지금 보시기엔 너무 어려운 내용이 많죠?

지금은 어려울 수도 있습니다. 머신러닝과 관련된 개념이 나오기 때문인데, 나중에 데이커 여러분들이 머신러닝 기초와 의사결정나무(Decision Tree), 랜덤포레스트(Random Forest) 등을 공부하시게 된다면 무슨 소리인지 이해하실 수 있을겁니다.

# 3. 모델링


In [9]:
## 분석 모집단 생성
# public, private에 필요한 데이터 날짜
pub_base_dt = '20211029'  
pri_base_dt = '20211126'  
## Public, Private 평가 기간
dl_predict_pub_day = ['2021-11-01', '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05']   
dl_predict_pri_day = ['2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03']   

## 분석 모집단 생성
df_mst = df_master.query("num >=101 & cnt > 0 ")	
# num은 여기서 인덱스를 의미하는데, 주가를 예측하기 위해선 하루의 데이터가 아니라, 수십일 전까지의 데이터도 필요하기 때문에 앞의 100개의 데이터는 배제.
dl_col_1 = ['dt', 'code', 'amt']     #날짜, 종목코드, 거래대금
dl_col_2 = df_mst.columns[pd.Series(df_mst.columns).str.startswith('y_')].values.tolist()   
#y_로 시작하는 (향후 5일치 주가 향방)값을 리스트로 불러오기.
dl_x1 = df_mst.columns[pd.Series(df_mst.columns).str.startswith('x_')].values.tolist()
#x_로 시작하는 값(환율과 주가지수)을 리스트로 불러온다.  
dl_x2 = ['end', 'avg_amt_20', 'end_usdkrw']  #종가, 거래대금 20일 이동평균, 종가
dl_x = dl_x1 + dl_x2 
df_mst = df_mst[dl_col_1 + dl_col_2 + dl_x]  # 분석에 필요한 컬럼만 추출해 df_mst에 저장.


## 학습 조건 설정
dl_look_after = [1, 2, 3, 4, 5]                        # 예측기간
dl_n_trin = [800, 400, 200, 100, 50]     # 학습 데이타건수
dl_target = ['y_rt_01d_eeup', 'y_rt_02d_eeup', 'y_rt_03d_eeup', 'y_rt_04d_eeup', 'y_rt_05d_eeup']   # Target Name
dl_yn     = ['y_01d_yn', 'y_02d_yn', 'y_03d_yn', 'y_04d_yn', 'y_05d_yn']                            # Target 불능 여부

df_m2_pre_c1_tot = pd.DataFrame()
df_m2_pre_c2_tot = pd.DataFrame()

## 예측기간별 모델링
for look in dl_look_after:

	print("\n ===> look_after : ",  look)
	
	df_mst_tt = df_mst.copy()

	n_trin_sample = dl_n_trin[look-1]   
	random.seed(123)
	
	str_target            = dl_target[look-1]      
	str_yn                = dl_yn[look-1]     
	look_after            = look      
	df_mst_tt.rename(columns = {str_target:'Target'}, inplace = True)    
	
	## 학습 및 예측 조건	
    # c1은 public을 예측하기 위한 데이터, c2는 private을 예측하기 위한 데이터
    # m1은 훈련용 데이터, m2는 예측용 데이터
	df_m1_tt_c1 = df_mst_tt[(df_mst_tt['dt'] >= '20120101') & (df_mst_tt['dt'] < pub_base_dt)]   	           
	df_m1_tt_c2 = df_mst_tt[(df_mst_tt['dt'] >= '20120101') & (df_mst_tt['dt'] < pri_base_dt)]   
	df_m2_tt_c1 = df_mst_tt[(df_mst_tt['dt'] == pub_base_dt)]  
	df_m2_tt_c2 = df_mst_tt[(df_mst_tt['dt'] == pri_base_dt)]  
	df_m1_tt_c1 = df_m1_tt_c1[df_m1_tt_c1[str_yn]==1]   
	df_m1_tt_c2 = df_m1_tt_c2[df_m1_tt_c2[str_yn]==1]   	
	df_m2_tt_c1 = df_m2_tt_c1[df_m2_tt_c1['code'].isin(stock_list['code'])] 
	df_m2_tt_c2 = df_m2_tt_c2[df_m2_tt_c2['code'].isin(stock_list['code'])] 
	
	## Target Scaling	
    # 이상치(너무 극단적인 데이터)를 제거하기 위해, 상위 20퍼에서 상위 80퍼 안에 들어가는 값만 추출
	target_out1 = df_m1_tt_c1['Target'].describe(percentiles = [0.2, 0.8])
	target_out2 = df_m1_tt_c2['Target'].describe(percentiles = [0.2, 0.8])
	df_m1_tt_c1 = df_m1_tt_c1[(df_m1_tt_c1['Target'] >= target_out1['20%']) & (df_m1_tt_c1['Target'] < target_out1['80%'])]
	df_m1_tt_c2 = df_m1_tt_c2[(df_m1_tt_c2['Target'] >= target_out2['20%']) & (df_m1_tt_c2['Target'] < target_out2['80%'])]
	

	def f_featuring(df_m1_tt, df_m2_tt):
		
		random.seed(123)   
		
		## Sampling		
        # 학습 데이터를 랜덤하게 추출하여 사용
		rows = random.sample(df_m1_tt.index.tolist(), n_trin_sample)     
		df_m1_sam = df_m1_tt.loc[rows]			
		df_m1_sam = df_m1_sam.sort_values(by=['dt'], axis=0)    		
		df_m2_sam = df_m2_tt.sort_values(by=['dt'], axis=0)   
		
		## X, Y 분리			
		dl_y = df_m1_sam.columns[pd.Series(df_m1_sam.columns).str.startswith('Target')].values.tolist()  			
		df_m1_x = df_m1_sam[dl_x]	
		df_m2_x = df_m2_sam[dl_x]	
		df_m1_y = df_m1_sam[dl_y]	
		df_m2_y = df_m2_sam[dl_y]
			
		## Missing 처리
		df_m1_x = df_m1_x.replace([np.inf, -np.inf], np.nan)	
		df_m2_x = df_m2_x.replace([np.inf, -np.inf], np.nan)
        # 뒤에서 로그변환을 해주어야 하는데, log(inf) = 0 이므로, 0이 되지 않게 변환 후 제거
        
		df_m1_x.dropna(how='any', axis=1, inplace=True)  
        # how='all'은 한 행이 가지는 모든 컬럼이 결측치여야 drop
		dl_tot_cols = df_m1_x.columns                    
		df_m2_x = df_m2_x[dl_tot_cols]                   
			
		dl_num_cols = df_m1_x._get_numeric_data().columns                    
		if len(dl_num_cols) >= 1:
			imp = SimpleImputer(missing_values=np.nan, strategy='median')
            # SimpleImputer는 결측치를 대체하는 기법 중 하나로, 숫자형(연속형) 데이터는 strategy = 'median'으로 지정하여 중앙값으로 대체
            
			imp = imp.fit(df_m1_x[dl_num_cols])
			df_m1_x[dl_num_cols] = imp.transform(df_m1_x[dl_num_cols])
			df_m2_x[dl_num_cols] = imp.transform(df_m2_x[dl_num_cols])
			
		dl_cls_cols = df_m1_x.select_dtypes(include=['object']).columns.tolist()		
		if len(dl_cls_cols) >= 1:		
            # 문자형 데이터는 결측치를 'missing'이라는 문자로 대체
			df_m1_x[dl_cls_cols] = df_m1_x[dl_cls_cols].fillna('missing')
			df_m2_x[dl_cls_cols] = df_m2_x[dl_cls_cols].fillna('missing')

		## Unique 변수 제외 -> 한 변수의 값이 하나밖에 없는 변수를 제거
		df_col_list = df_m1_x.apply(pd.Series.nunique) != 1    
		df_m1_x = df_m1_x.loc[:, df_col_list]
		df_m2_x = df_m2_x.loc[:, df_col_list]				

		## 연속형 변수 Log 변환 		
        # 로그변환은 큰 수를 작게 만들거나, 복잡한 계산을 간편하게 할 경우 사용, 왜도와 첨도를 줄여서 데이터 분석 시 의미있는 결과를 도출하기 위함.
		dl_num_cols = df_m1_x._get_numeric_data().columns   
		for col in dl_num_cols:		  
			df_m1_x[col] = np.where(df_m1_x[col] < 0, -np.log(1 + -df_m1_x[col]), np.log(1 + df_m1_x[col]))   
			df_m2_x[col] = np.where(df_m2_x[col] < 0, -np.log(1 + -df_m2_x[col]), np.log(1 + df_m2_x[col]))   
					
		## 변수 표준화 
        # 연속형 데이터는 범주형 데이터와 달리 정규화 과정이 필요. 대부분의 알고리즘들이 변수의 범위에 영향을 받기 때문에 각 변수가 가지는 값들의 숫자 범위가 다를 경우, 이를 일정한 범위로 맞춰줘야함.
		dl_num_cols = df_m1_x._get_numeric_data().columns   
		scaler = StandardScaler().fit(df_m1_x[dl_num_cols])           
		df_m1_x[dl_num_cols] = scaler.transform(df_m1_x[dl_num_cols])  
		df_m2_x[dl_num_cols] = scaler.transform(df_m2_x[dl_num_cols])  
		
		## One Hot Encoding 		
        # 원-핫 인코딩은 단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 값의 인덱스에 1의 값을 부여하고, 다른 인덱스에는 0을 부여하는 데이터터의 벡터 표현 방식. 
		dl_cls_cols = df_m1_x.select_dtypes(include=['object']).columns.tolist()		
		df_m1_x_dumy = pd.get_dummies(df_m1_x[dl_cls_cols], prefix=dl_cls_cols)   
		df_m2_x_dumy = pd.get_dummies(df_m2_x[dl_cls_cols], prefix=dl_cls_cols)
		dl_col_m1 = df_m1_x_dumy.columns	
		dl_col_m2 = df_m2_x_dumy.columns

		dl_col_m1_m2 = list(set(dl_col_m1) - set(dl_col_m2))  
		if len(dl_col_m1_m2) >= 1:
			for col in dl_col_m1_m2:
				df_m2_x_dumy[col] = 0					

		df_m1_x = pd.concat([df_m1_x[dl_num_cols], df_m1_x_dumy], axis=1)
		df_m2_x = pd.concat([df_m2_x[dl_num_cols], df_m2_x_dumy], axis=1)	
		df_m1_x, df_m2_x = df_m1_x.align(df_m2_x, join='left', axis=1, fill_value=0)

		## 상관분석
        # 상관계수가 매우 높은(절댓값이 0.95보다 큰) 컬럼을 제거.
		corr_matrix = df_m1_x.corr()		
		df_corr_upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))  
		to_drop = [column for column in df_corr_upper.columns if (any(df_corr_upper[column] >= 0.95) | any(df_corr_upper[column] < -0.95))]   
		df_m1_x.drop(to_drop, axis=1, inplace=True)
		df_m2_x.drop(to_drop, axis=1, inplace=True)
		
		## Feature 안정성 분석		
		if len(df_m1_x.columns) > 120:   
			num = 120                    
		else:
			num = len(df_m1_x.columns)   
		model = SelectKBest(f_regression, k= num) 
# SelectKBest는 데이터에 컬럼이 너무 많은 경우 모델의 설명력을 가장 높일 수 있는 K개의 컬럼만 선택하게 하는 모듈.
# 데이터에 컬럼이 너무 많을 경우 모델 학습시간이 오래걸리기도 하고, 과대적합이라는 문제가 발생.        
# f_regression은 회귀분석의 F-test을 진행하는 모듈. SelectKBest에서 최적의 변수 조합을 찾아내기 위해 사용.
		col_list = pd.DataFrame(df_m1_x.columns, columns=['col'])
		int_num = int(len(df_m1_x)/3)
		fit = model.fit(df_m1_x[:int_num], df_m1_y[:int_num])
		score_list_1 = pd.DataFrame(model.scores_, columns=['score'])
		fit = model.fit(df_m1_x[int_num:int_num*2], df_m1_y  [int_num:int_num*2])
		score_list_2 = pd.DataFrame(model.scores_, columns=['score'])
		fit = model.fit(df_m1_x[int_num*2:], df_m1_y[int_num*2:])
		score_list_3 = pd.DataFrame(model.scores_, columns=['score'])	
		df_importance = pd.concat([col_list, score_list_1, score_list_2, score_list_3], axis=1)     
		df_importance['mean'] = df_importance.iloc[:, [1, 2, 3]].min(1)
		df_importance['rank'] = df_importance['mean'].rank(ascending=0)  
		df_m1_x = df_m1_x[df_importance.loc[df_importance['rank'] <= num, 'col']]
		df_m2_x = df_m2_x[df_importance.loc[df_importance['rank'] <= num, 'col']]		
						
		return df_m1_x, df_m2_x, df_m1_y, df_m2_y, df_m2_sam
		
	df_m1_x_c1, df_m2_x_c1, df_m1_y_c1, df_m2_y_c1, df_m2_sam_c1 = f_featuring(df_m1_tt_c1, df_m2_tt_c1)
	df_m1_x_c2, df_m2_x_c2, df_m1_y_c2, df_m2_y_c2, df_m2_sam_c2 = f_featuring(df_m1_tt_c2, df_m2_tt_c2)

	## Model Fitting		
	gbl = globals()
    # globals() 함수는 특정 모듈내에서 사용되는 전역 변수들의 변수 이름 및 현재 값 등을 알고 싶을 때 사용하는 함수. 
    # globals 함수는 코드내의 전역 변수의 이름과 현재 값을 출력. 
    # 여기서 전역변수란, 지역변수랑 반대되는 개념으로 하나의 반복문이나 조건문, 사용자정의함수 등에서 변수를 사용할 때 해당 변수를 구문 밖에서도 사용할 수 있는 변수를 의미.
    
    
	def f_modeling_1(case_flg):

		df_m1_x = gbl['df_m1_x_' + case_flg]
		df_m2_x = gbl['df_m2_x_' + case_flg]
		df_m1_y = gbl['df_m1_y_' + case_flg]
		df_m2_y = gbl['df_m2_y_' + case_flg]	
		df_m2_sam = gbl['df_m2_sam_' + case_flg]
		train_data = lgb.Dataset(df_m1_x, label = df_m1_y)	
		parameters = {
			'n_leaves':10,
			'n_estimators':80, 
			'max_depth':-1,  
			'learning_rate':0.01, 
			'subsample':1,
			'colsample_bytree':0.8,
			'reg_alpha':0.1,
			'reg_lambda':1,
			'objective': 'regression',
			'min_data':1,
			'min_data_in_bin':1,	
			'random_state' : 123} 			
        # 인자 설명은 앞선 2장에서 다루었습니다.
        
		fit = lgb.train(parameters, train_data, num_boost_round=5000)	
		df_m1_pre = pd.DataFrame(fit.predict(df_m1_x))
		df_m2_pre = pd.DataFrame(fit.predict(df_m2_x))
							
		## Predict
		df_m2_pre.rename(columns = {0:"score"}, inplace = True)
		df_m2_pre = pd.concat([df_m2_pre.reset_index(drop=True), 
							   df_m2_sam[['Target', 'dt', 'code', 'amt', 'end']].reset_index(drop=True)], axis=1)		
		df_m2_pre['end_a1'] = df_m2_pre['end'] * df_m2_pre['Target']			
		df_m2_pre['end_a1_pred'] = df_m2_pre['end'] * df_m2_pre['score'] 
		df_m2_pre['model'] = 'M1' 
		df_m2_pre['case'] = case_flg
		df_m2_pre['look_after'] = look_after

		return df_m2_pre

	df_m2_pre_c1 = f_modeling_1('c1')       
	df_m2_pre_c2 = f_modeling_1('c2')	
	df_m2_pre_c1_tot = df_m2_pre_c1_tot.append(df_m2_pre_c1)
	df_m2_pre_c2_tot = df_m2_pre_c2_tot.append(df_m2_pre_c2)


 ===> look_after :  1
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002100 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29841
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 120
[LightGBM] [Info] Start training from score 1.000430
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000600 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29841
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 120
[LightGBM] [Info] Start training from score 1.000730

 ===> look_after :  2
[Lig


**여기서 잠깐!** ✋


> [Warning] No further splits with positive gain, best gain: -inf 


이라는 경고문구가 많이 뜰 텐데요. 원래 위 코드는 모든 주식종목의 데이터를 활용하는 코드인데, 우리는 삼성전자의 데이터만 다루고 있기 때문에 데이터가 부족해서
위와 같은 경고문구가 뜰 것입니다. 무시하고 넘어가셔도 좋아요 :)

이상으로 ‘모델 소개 및 학습’ 주제 Stage 5이였습니다.
마지막 Stage인 Stage 6에는 최종적으로 예측된 데이터를 후처리 해주고, 제출 파일까지 만드는 내용으로 마무리 짓도록 하겠습니다.

다음에 또 만나요!🫶